In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

E0412 23:09:39.883289375    1420 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0412 23:09:42.751047508    1420 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
## parameters cell
itp_id = 300

In [3]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [4]:
# airtable_organizations

In [5]:
analysis_date = dt.date(2022, 3, 30)

In [6]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [7]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [8]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [9]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [10]:
rt_day.rt_trips

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,median_time,direction,mean_speed_mph,calitp_agency_name
0,300,0,2022-03-30,8295247101027228283,882164,3486,1,25902,2022-03-29,2099-01-01,8,19:30:05,Southbound,10.073085,Big Blue Bus
2,300,0,2022-03-30,5645703968817423876,882134,3486,1,25902,2022-03-29,2099-01-01,8,17:36:10,Southbound,6.216009,Big Blue Bus
3,300,0,2022-03-30,1687359312824429981,882102,3486,0,25899,2022-03-29,2099-01-01,8,15:53:27,Northbound,9.774798,Big Blue Bus
4,300,0,2022-03-30,430295924079312381,882153,3486,1,25902,2022-03-29,2099-01-01,8,07:44:49,Southbound,11.008924,Big Blue Bus
5,300,0,2022-03-30,1054074058611617401,882165,3486,1,25902,2022-03-29,2099-01-01,8,19:50:04,Southbound,8.870510,Big Blue Bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,300,0,2022-03-30,7348749684362675513,883608,3500,1,25954,2022-03-29,2099-01-01,43,06:51:50.500000,Southbound,18.074227,Big Blue Bus
1511,300,0,2022-03-30,1141210565827503493,883575,3500,0,25953,2022-03-29,2099-01-01,43,17:56:20,Westbound,8.695437,Big Blue Bus
1513,300,0,2022-03-30,7800219397066221136,883604,3500,1,25954,2022-03-29,2099-01-01,43,08:48:22,Southbound,14.365133,Big Blue Bus
1514,300,0,2022-03-30,-6353648903219724979,883576,3500,0,25953,2022-03-29,2099-01-01,43,17:32:24,Westbound,7.659799,Big Blue Bus


# Bus Speed Maps and Delay Analysis

In [20]:
from IPython.display import display, Markdown, Latex

In [24]:
Markdown(f'{rt_day.calitp_agency_name}, District {airtable_organizations.caltrans_district.iloc[0]}')

### Big Blue Bus, District 07 - Los Angeles

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m

In [ ]:
%%capture
rt_day.set_filter(start_time='07:00', end_time='09:00')

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m

In [ ]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance)
         >> arrange(-_.speed_variance)
         >> head(20)
        )
    metrics_df
except Exception as e:
    print(e)
## check that route has all-day span...

In [ ]:
def map_from_metrics(rt_day, metrics_df):
    try:
        route_name = metrics_df.iloc[1,0]
        direction_id = metrics_df.iloc[1,1]
    except Exception as e:
        print(e)
        return
    # route_name = '281'
    # direction_id = '1'
    rt_day.set_filter(start_time='07:00', end_time='09:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        am_map = rt_day.segment_speed_map(how='low_speeds')
        display(am_map)
    except Exception as e:
        print(e)
        pass
    
    rt_day.set_filter(start_time='11:00', end_time='14:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        mid_map = rt_day.segment_speed_map(how='low_speeds')
        display(mid_map)
    except Exception as e:
        print(e)
        pass

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        pm_map = rt_day.segment_speed_map(how='low_speeds')
        display(pm_map)
    except Exception as e:
        print(e)
        pass

    return

In [ ]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    print(e)
    pass